# Call Analytics with Amazon Bedrock

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Overview

In this example, you will use Amazon Bedrock to summarize and analyze the quality of a transcript of a customer service call using an LLM. A sample transcript is provided in the './data/' folder to get started. You can also use your own transcript file. This example utilizes Amazon Bedrock, Anthropic Claude 2.0 large language model (LLM), LangChain framework, and Pydantic parser to summarize and analyze the quality of customer service call transcripts. 

### Use case

XYZ is a travel booking and vacation experience company. Customers can contact XYZ customer service representatives (CSRs) by phone to book new trips or modify existing reservations. These customer-CSR conversations are recorded, transcribed after the call ends, and analyzed by the call center management team to improve customer service processes and policies. The call transcription and analysis approach is also applicable for assessing customer interactions via email, chat, and other mediums.

### How does this work?

Transcript Ingestion
- Customer service call transcripts are uploaded as JSON objects to an S3 bucket. Plain text formats are also supported.  

Summarization Workflow
- The transcript is retrieved from the S3 source bucket and fed as a prompt to Claude, an AI assistant from Anthropic.  
- Leveraging natural language capabilities, Claude analyzes the dialogue and returns a JSON summary highlighting key discussion points and outcomes. This step uses LangChain for natural language processing and Pydantic for output structuring.

Quality Analysis Workflow  
- The full transcript is also analyzed by Claude against pre-defined quality criteria such as issue resolution, adherence to process standards, etc.  
- Assessment results are returned by Claude in a JSON format conforming to Pydantic schemas. Powered by LangChain for natural language comprehension and Pydantic for output formatting.

The two independent workflows allow simultaneous call summarization and quality analysis based on a single transcript ingestion event.

![](./images/call-analytics-example.png)


## Setup Environment

In [1]:
#Install LangChain
%pip install langchain==0.1.16 -Uq
%pip install rich -Uq
#%pip install langchain

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [54]:
from enum import Enum
from functools import partial
from pathlib import Path
from typing import List, Dict
import boto3
import json
import re

from pydantic import BaseModel, Field
from rich import print as rprint
from rich.markdown import Markdown

from langchain.chat_models import BedrockChat
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import AIMessage
from langchain.output_parsers import PydanticOutputParser

# Create Bedrock and S3 clients
boto3_session = boto3.session.Session()
bedrock_runtime = boto3_session.client("bedrock-runtime")
s3 = boto3.client("s3")

# Define LLM model
# llm_modelId = "anthropic.claude-v2"
# llm_modelId = "anthropic.claude-v2:1"
llm_modelId = "anthropic.claude-3-sonnet-20240229-v1:0"

llm = BedrockChat(
    model_id=llm_modelId,
    model_kwargs={
        "max_tokens": 8000,
        "stop_sequences": ["\n\nHuman:"],
        "temperature": 0.4,
        "top_p": 1,
    },
    client=bedrock_runtime,
)

# Define S3 Location for customer service calls
s3_bucket = "your-bucket-name"

if s3_bucket == "your-bucket-name":
    print("WARNING: PLease update 's3_bucket' with a name of your S3 bucket.")
s3_key_transcripts = "call-analytics/call-transcript/"
s3_key_summary = "call-analytics/call-summary/"
s3_key_score = "call-analytics/call-score/"

# Define call transcript file name
call_transcript_file = "Call Transcript Sample 1.json"

pretty_print_json = lambda x: rprint(Markdown(f"```json\n{json.dumps(json.loads(x), indent=2)}\n```"))

In [55]:
# Read call transcript file either from the repository or from an S3 bucket

##########################################################################
# OPTION 1: read from ./data/ folder in the repository (default)
##########################################################################
transcripts = Path("data")
transcript = (transcripts / call_transcript_file).open("r").read()

##########################################################################
# OPTION 2: read from S3 Location for customer service calls defined above
##########################################################################
#obj = s3.get_object(Bucket=s3_Bucket, Key=s3_Key_Transcripts + call_transcript_file)
#transcript = obj['Body'].read()

# Define variables to simplify code in subsequent steps 
transcript_dict = json.loads(transcript)
call_date = json.loads (transcript)['call_date']
call_time = json.loads (transcript)['call_time']

pretty_print_json(json.dumps(transcript_dict))

                                                                                                                   
 {                                                                                                                 
   "call_ID": "12345",                                                                                             
   "CSR_ID": "JaneDoe123",                                                                                         
   "call_date": "2024-02-01",                                                                                      
   "call_time": "02:16:43",                                                                                        
   "call_transcript": [                                                                                            
     "CSR: Thank you for calling ABC Travel, this is Jane. How may I assist you today? ",                          
     "Customer: Yes, I need help with a reservation I made last week. This is unacceptable service! ",             
     "CSR: I apologize for the trouble. May I have your name and reservation number to look up your booking? ",    
     "Customer: It's John Smith. My reservation number is 012345. I booked a trip to Hawaii last week and just got 
 an email that my flight was canceled! This is ridiculous. ",                                                      
     "CSR: Let me take a look at your reservation here Mr. Smith. I see that your flight from Chicago to Honolulu  
 March 15th was indeed canceled by the airline. I do apologize for this inconvenience. ",                          
     "Customer: This is unbelievable! I booked this trip months ago. How could you just cancel my flight like that 
 I took time off work and made so many plans. This is completely unacceptable! ",                                  
     "CSR: You're absolutely right, having a flight canceled can be very disruptive. As your travel agent, I want  
 do everything I can to get this fixed for you right away. It looks like the airline has rebooked you on a flight  
 that leaves a few hours later on the same day. I know that's still an inconvenience though. Let me see what other 
 options may be available. ",                                                                                      
     "Customer: This is ridiculous. I should get a full refund if you're going to cancel my flight like that. I    
 don't want another flight, I just want my money back! ",                                                          
     "CSR: I completely understand your frustration, Mr. Smith. Since this cancellation was initiated by the       
 airline, you are entitled to a full refund if you prefer not to be rebooked. I can definitely process that refund 
 for the flight cost right away. How about the hotel and other portions of your trip - would you like for me to lo 
 into refunds or changes for those as well? My goal is to make sure you are completely satisfied. ",               
     "Customer: This is unacceptable. I spent so much money on this trip and now it's ruined. I want a full refund 
 for everything - the flight, the hotel, the car rental. You need to fix this! ",                                  
     "CSR: You're absolutely right, Mr. Smith. Let me process full refunds for your entire trip booking right now. 
 see you booked 2 roundtrip flights, 5 nights hotel in Honolulu, and a 7 day car rental. I will get all of those   
 refunded in full immediately. You should see the refund hit your credit card in 3-5 business days. I sincerely    
 apologize that we had to cancel a portion of your trip. Providing a seamless travel experience is our top priorit 
 so I appreciate you bringing this issue to my attention. ",                                                       
     "Customer: How could you let this happen? I booked my trip so far in advance specifically to avoid problems!  
 Now everything is ruined and I had to waste my time cal

## Call Summarization

In this step, we utilize the LangChain framework and Pydantic parser to generate LLM output in JSON format.

Refer to [Pydantic parser](https://python.langchain.com/docs/modules/model_io/output_parsers/types/pydantic) on LangChain site for addtional details and examples. 


We will ask Claude to generate a json output and place it into a specified XML tag. We'll then use the function below to extract just the json output.

In [15]:
def extract_from_xml_tag(response:AIMessage, tag:str) -> str:
    response = response.content
    tag_txt = re.search(rf'<{tag}>(.*?)</{tag}>', response, re.DOTALL)
    if tag_txt:
        return tag_txt.group(1)
    else:
        print(response)
        return ""

In [21]:
# Define a data schema for the LLM output with required attributes
class CallSummary(BaseModel):
    call_summary: str = Field(description="Call transcript summary: ")
    key_takeaways: List[str] = Field(description="Call transcript key takeaways: ")
    follow_up_actions: List[str] = Field(
        description="Call Transcript key action items: "
    )


# Define Pydantic parser based on the data schema
summarization_parser = PydanticOutputParser(pydantic_object=CallSummary)

# Define a template for the LLM prompt with {format_instructions} and  {transcript} placeholder inputs
summarization_template = """

Please provide a summary of the following call transcript provided between <transcript></transcript> tags. 
Capture key takeaways and specific follow up actions. 
Skip the preamble and go straight to the answer.

<transcript>{transcript}</transcript>

Format your response per the instructions below: 
{format_instructions} 

Place your response between <output></output> tags. 
"""

# Incorporate the format instructions into the LLM prompt based on the prompt template
summarization_prompt = ChatPromptTemplate.from_template(
    summarization_template,
    partial_variables={
        "format_instructions": summarization_parser.get_format_instructions()
    },
)

In [20]:
# Review the content of the prompt with format instructions (notice the JSON instructions generated by Pydantic parser)
rprint (summarization_prompt.dict())

{
    'name': None,
    'input_variables': ['transcript'],
    'input_types': {},
    'output_parser': None,
    'partial_variables': {},
    'metadata': None,
    'tags': None,
    'messages': [
        {
            'prompt': {
                'name': None,
                'input_variables': ['transcript'],
                'input_types': {},
                'output_parser': None,
                'partial_variables': {
                    'format_instructions': 'The output should be formatted as a JSON instance that conforms to the 
JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list 
of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", 
"baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not 
well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"call_summary": {"description": "Call 
transcript summary: ", "title": "Call Summary", "type": "string"}, "key_takeaways": {"description": "Call 
transcript key takeaways: ", "items": {"type": "string"}, "title": "Key Takeaways", "type": "array"}, 
"follow_up_actions": {"description": "Call Transcript key action items: ", "items": {"type": "string"}, "title": 
"Follow Up Actions", "type": "array"}}, "required": ["call_summary", "key_takeaways", "follow_up_actions"]}\n```'
                },
                'metadata': None,
                'tags': None,
                'template': '\n\nPlease provide a summary of the following call transcript provided between 
<transcript></transcript> tags. \nCapture key takeaways and specific follow up actions. \nSkip the preamble and go 
straight to the answer.\n\n<transcript>{transcript}</transcript>\n\nFormat your response per the instructions 
below: \n{format_instructions} \n\nPlace your response between <output></output> tags. \n',
                'template_format': 'f-string',
                'validate_template': False,
                '_type': 'prompt'
            },
            'additional_kwargs': {}
        }
    ],
    'validate_template': False,
    '_type': 'chat'
}

In [22]:
# Define a function to convert the call transcript text from 'list' to 'string' format
def process_transcript(transcript: str) -> str:
    json_transcript = json.loads(transcript)
    call_transcript = "\n".join(json_transcript.get("call_transcript", []))

    return call_transcript


# Construct the chain by assembling all required components using LangChain '|' operator

summarization_chain = (
    {"transcript": RunnableLambda(process_transcript)}               # process the call transcript using the function above
    | summarization_prompt                                           # inject the processed call transcript into the LLM prompt
    | llm                                                            # send the prompt to the LLM 
    | RunnableLambda(partial(extract_from_xml_tag, tag="output"))    # extract the JSON string from <output> tag in the LLM response
    | summarization_parser                                           # validate the JSON string using the Pydantic parser
)

In [56]:
pretty_print_json(transcript)

                                                                                                                   
 {                                                                                                                 
   "call_ID": "12345",                                                                                             
   "CSR_ID": "JaneDoe123",                                                                                         
   "call_date": "2024-02-01",                                                                                      
   "call_time": "02:16:43",                                                                                        
   "call_transcript": [                                                                                            
     "CSR: Thank you for calling ABC Travel, this is Jane. How may I assist you today? ",                          
     "Customer: Yes, I need help with a reservation I made last week. This is unacceptable service! ",             
     "CSR: I apologize for the trouble. May I have your name and reservation number to look up your booking? ",    
     "Customer: It's John Smith. My reservation number is 012345. I booked a trip to Hawaii last week and just got 
 an email that my flight was canceled! This is ridiculous. ",                                                      
     "CSR: Let me take a look at your reservation here Mr. Smith. I see that your flight from Chicago to Honolulu  
 March 15th was indeed canceled by the airline. I do apologize for this inconvenience. ",                          
     "Customer: This is unbelievable! I booked this trip months ago. How could you just cancel my flight like that 
 I took time off work and made so many plans. This is completely unacceptable! ",                                  
     "CSR: You're absolutely right, having a flight canceled can be very disruptive. As your travel agent, I want  
 do everything I can to get this fixed for you right away. It looks like the airline has rebooked you on a flight  
 that leaves a few hours later on the same day. I know that's still an inconvenience though. Let me see what other 
 options may be available. ",                                                                                      
     "Customer: This is ridiculous. I should get a full refund if you're going to cancel my flight like that. I    
 don't want another flight, I just want my money back! ",                                                          
     "CSR: I completely understand your frustration, Mr. Smith. Since this cancellation was initiated by the       
 airline, you are entitled to a full refund if you prefer not to be rebooked. I can definitely process that refund 
 for the flight cost right away. How about the hotel and other portions of your trip - would you like for me to lo 
 into refunds or changes for those as well? My goal is to make sure you are completely satisfied. ",               
     "Customer: This is unacceptable. I spent so much money on this trip and now it's ruined. I want a full refund 
 for everything - the flight, the hotel, the car rental. You need to fix this! ",                                  
     "CSR: You're absolutely right, Mr. Smith. Let me process full refunds for your entire trip booking right now. 
 see you booked 2 roundtrip flights, 5 nights hotel in Honolulu, and a 7 day car rental. I will get all of those   
 refunded in full immediately. You should see the refund hit your credit card in 3-5 business days. I sincerely    
 apologize that we had to cancel a portion of your trip. Providing a seamless travel experience is our top priorit 
 so I appreciate you bringing this issue to my attention. ",                                                       
     "Customer: How could you let this happen? I booked my trip so far in advance specifically to avoid problems!  
 Now everything is ruined and I had to waste my time cal

In [57]:
# Invoke the chain; the output will contain the LLM output in JSON format
summary = summarization_chain.invoke(transcript)

pretty_print_json(summary.json())

                                                                                                                   
 {                                                                                                                 
   "call_summary": "The customer, John Smith, was upset because his flight to Hawaii was canceled by the airline.  
 The customer service representative, Jane, offered to rebook him on another flight or provide a full refund for t 
 trip. After some back-and-forth, the customer demanded a full refund for the flight, hotel, and car rental. Jane  
 processed the refunds and offered a $200 travel voucher, but the customer remained unsatisfied and requested to   
 speak with a supervisor. The supervisor, Sarah, apologized and took responsibility for the poor service. She      
 offered to look into additional compensation to restore the customer's confidence.",                              
   "key_takeaways": [                                                                                              
     "The customer's flight to Hawaii was canceled by the airline, disrupting his travel plans",                   
     "The customer service representative offered to rebook or provide a full refund",                             
     "After demanding a full refund, the customer remained unsatisfied with the service",                          
     "The supervisor took responsibility and offered to investigate further to regain the customer's trust"        
   ],                                                                                                              
   "follow_up_actions": [                                                                                          
     "Process refunds for the customer's entire trip booking",                                                     
     "Assess what went wrong and implement better training around managing cancellations and rebookings",          
     "Follow up with the customer to offer additional compensation or incentives to restore confidence"            
   ]                                                                                                               
 }                                                                                                                 
                                                                                                                   

In [58]:
# Extract the required values for preview: Call Summary, Key Takeaways, Follow Up Actions.
call_summary = summary.call_summary
key_takeaways = "-" + "\n-".join(summary.key_takeaways)
follow_up_actions = "-" + "\n-".join(summary.follow_up_actions)

rprint(
    f"Call Summary:\n{call_summary}\n\nKey Takeaways:\n{key_takeaways}\n\nFollow Up Actions\n{follow_up_actions}"
)

Call Summary:
The customer, John Smith, was upset because his flight to Hawaii was canceled by the airline. The customer service 
representative, Jane, offered to rebook him on another flight or provide a full refund for the trip. After some 
back-and-forth, the customer demanded a full refund for the flight, hotel, and car rental. Jane processed the 
refunds and offered a $200 travel voucher, but the customer remained unsatisfied and requested to speak with a 
supervisor. The supervisor, Sarah, apologized and took responsibility for the poor service. She offered to look 
into additional compensation to restore the customer's confidence.

Key Takeaways:
-The customer's flight to Hawaii was canceled by the airline, disrupting his travel plans
-The customer service representative offered to rebook or provide a full refund
-After demanding a full refund, the customer remained unsatisfied with the service
-The supervisor took responsibility and offered to investigate further to regain the customer's trust

Follow Up Actions
-Process refunds for the customer's entire trip booking
-Assess what went wrong and implement better training around managing cancellations and rebookings
-Follow up with the customer to offer additional compensation or incentives to restore confidence

In [59]:
# Construct call summary as JSON object with all relevant attributes to be stored in S3
bedrock_response = json.loads(summary.json())
bedrock_response ["call_ID"] = transcript_dict['call_ID']
bedrock_response ["CSR_ID"] = transcript_dict['CSR_ID']
bedrock_response ["call_date"] = call_date
bedrock_response ["call_time"] = call_time
bedrock_response ["llm_model"] = llm_modelId
bedrock_response = json.dumps (bedrock_response, indent=2)
pretty_print_json(bedrock_response)

# Write Bedrock output text to S3 object  
if s3_bucket != "your-bucket-name":
    s3_key = s3_key_summary + "Call Summary " + call_date + " " + call_time + ".json"
    s3.put_object(Body=bedrock_response, Bucket=s3_bucket, Key=s3_key )
    print("Transcript summary written to S3:" + s3_key)

else:
    rprint("[magenta bold]WARNING: No S3 bucket defined. Transcript summary not written to S3.[/magenta bold]")

                                                                                                                   
 {                                                                                                                 
   "call_summary": "The customer, John Smith, was upset because his flight to Hawaii was canceled by the airline.  
 The customer service representative, Jane, offered to rebook him on another flight or provide a full refund for t 
 trip. After some back-and-forth, the customer demanded a full refund for the flight, hotel, and car rental. Jane  
 processed the refunds and offered a $200 travel voucher, but the customer remained unsatisfied and requested to   
 speak with a supervisor. The supervisor, Sarah, apologized and took responsibility for the poor service. She      
 offered to look into additional compensation to restore the customer's confidence.",                              
   "key_takeaways": [                                                                                              
     "The customer's flight to Hawaii was canceled by the airline, disrupting his travel plans",                   
     "The customer service representative offered to rebook or provide a full refund",                             
     "After demanding a full refund, the customer remained unsatisfied with the service",                          
     "The supervisor took responsibility and offered to investigate further to regain the customer's trust"        
   ],                                                                                                              
   "follow_up_actions": [                                                                                          
     "Process refunds for the customer's entire trip booking",                                                     
     "Assess what went wrong and implement better training around managing cancellations and rebookings",          
     "Follow up with the customer to offer additional compensation or incentives to restore confidence"            
   ],                                                                                                              
   "call_ID": "12345",                                                                                             
   "CSR_ID": "JaneDoe123",                                                                                         
   "call_date": "2024-02-01",                                                                                      
   "call_time": "02:16:43",                                                                                        
   "llm_model": "anthropic.claude-3-sonnet-20240229-v1:0"                                                          
 }                                                                                                                 
                                                                                                                   

WARNING: No S3 bucket defined. Transcript summary not written to S3.

## Call Quality Assessment
In this step, we utilize the LangChain framework and Pydantic parser to generate LLM output in JSON format. However, we expand the formatting instructions and prompt complexity in comparison to the Call Summarization step to achieve more tailored results. Specificially:

1. The prompt template provides a list of call quality assessment categories, each with a descriptive explanation of what should be evaluated within that category. 

2. The data model for the LLM output has two levels of nesting to capture detailed scoring for each category. 

Refer to [Pydantic parser](https://python.langchain.com/docs/modules/model_io/output_parsers/types/pydantic) on LangChain site for addtional details and examples. 

In [39]:
assessment_template = """
Evaluate call transcript against categories shown between <categories></categories> tags and provide score as 'High', 'Medium', 'Low' for each category.
Skip the preamble and go straight to the answer.

<categories>
1. Communication Skills:
 - Clarity: How clearly and concisely does the CSR communicate information?
 - Active Listening: Does the CSR actively listen to the customer's concerns and questions?
 - Empathy: How well does the CSR demonstrate empathy and understanding towards the customer?

2. Problem Resolution:
 - Effectiveness: How well did the CSR resolve the customer's issue or answer their question?
 - Timeliness: Was the issue resolved in a reasonable amount of time?

3. Product Knowledge:
 - Familiarity: Does the CSR have a good understanding of the company's products and services?
 - Accuracy: How accurate and precise are the answers provided by the CSR?

4. Professionalism:
 - Tone and Manner: How professional is the tone and manner of the CSR throughout the call?
 - Courtesy: Does the CSR maintain a courteous and respectful attitude towards the customer?

5. Problem Escalation:
 - Recognition: Did the CSR recognize when an issue required escalation to a higher level of support?
 - Handoff: How smoothly and effectively did the CSR transfer the call if escalation was necessary?

6. Resolution Follow-Up:
 - Follow-Up: Did the CSR provide information about any follow-up actions that would be taken?
 - Customer Satisfaction: Did the CSR inquire about the customer's satisfaction with the resolution?

7. Efficiency:
 - Call Handling Time: Was the call resolved efficiently without unnecessary delays?
 - Multi-Tasking: If applicable, did the CSR effectively handle multiple tasks during the call?

8. Adherence to Policies and Procedures:
 - Compliance: Did the CSR follow company policies and procedures in addressing the customer's issue?
 - Accuracy in Information: How well did the CSR adhere to the correct processes?

9. Technical Competence:
 - System Use: Did the CSR effectively navigate and use the customer service tools and systems?
 - Troubleshooting: How adept is the CSR at troubleshooting technical issues?

10. Customer Satisfaction:
 - Overall Satisfaction: How satisfied is the customer with the service received during the call?
 - Feedback: Did the CSR encourage the customer to provide feedback on the service?

11. Language Proficiency:
 - Clarity of Language: Was the language used by the CSR easily understandable?
 - Language Appropriateness: Did the CSR use appropriate language for effective communication?

12. Conflict Resolution:
 - Handling Difficult Customers: How well did the CSR manage and resolve conflicts with upset or frustrated customers?
 - De-escalation Skills: Did the CSR employ de-escalation techniques when needed?
<categories>

Here is the call transcript:
<transcript>{transcript}</transcript>

Format your response per the instructions below: 
{format_instructions} 

Place your response between <output></output> tags. 

"""

In [60]:
# Define a data schema for the LLM output with required attributes

class ScoreValue(Enum):
    High = "High"
    Medium = "Medium"
    Low = "Low"

class Score(BaseModel):
    score: ScoreValue
    score_explanation: str

class Evaluation(BaseModel):
    Communication_Skills: Score
    Problem_Resolution: Score
    Product_Knowledge: Score
    Professionalism: Score
    Problem_Escalation: Score
    Resolution_Follow_Up: Score
    Efficiency: Score
    Adherence_to_Policies_and_Procedures: Score
    Technical_Competence: Score
    Customer_Satisfaction: Score
    Language_Proficiency: Score
    Conflict_Resolution: Score
    
# Define Pydantic parser based on data schema 
assessment_parser = PydanticOutputParser(pydantic_object=Evaluation)

In [61]:
# Incorporate the format instructions into the LLM prompt based on the prompt template
assessment_prompt = ChatPromptTemplate.from_template(
    assessment_template,
    partial_variables={
        "format_instructions": assessment_parser.get_format_instructions()
    },
)

# Construct the chain by essembing all required components via LangChain '|' operator
assessment_chain = (
    {"transcript": RunnableLambda(process_transcript)}
    | assessment_prompt
    | llm
    | RunnableLambda(partial(extract_from_xml_tag, tag="output"))
    | assessment_parser
)

In [62]:
# Invoke the chain; the output will contain the LLM output in JSON format
call_assessment = assessment_chain.invoke(transcript)

In [64]:
# Preview score values for each category provided in the LLM output
for category, score in call_assessment:
    print(f"{category}: score={score.score.value}, explanation={score.score_explanation}\n")

Communication_Skills: score=High, explanation=The CSR communicated clearly, listened actively to the customer's concerns, and demonstrated empathy throughout the call.

Problem_Resolution: score=High, explanation=The CSR effectively resolved the customer's issue by processing full refunds for the canceled trip in a timely manner.

Product_Knowledge: score=High, explanation=The CSR demonstrated a good understanding of the company's policies and procedures regarding flight cancellations and refunds.

Professionalism: score=High, explanation=The CSR maintained a professional and courteous tone throughout the call, even when the customer was upset.

Problem_Escalation: score=High, explanation=The CSR recognized when the issue required escalation to a supervisor and transferred the call smoothly.

Resolution_Follow_Up: score=Medium, explanation=The CSR did not explicitly mention any follow-up actions, but did inquire about the customer's satisfaction with the resolution.

Efficiency: score=

In [65]:
# Preview content of the call_assessment JSON object
pretty_print_json(call_assessment.json())

                                                                                                                   
 {                                                                                                                 
   "Communication_Skills": {                                                                                       
     "score": "High",                                                                                              
     "score_explanation": "The CSR communicated clearly, listened actively to the customer's concerns, and         
 demonstrated empathy throughout the call."                                                                        
   },                                                                                                              
   "Problem_Resolution": {                                                                                         
     "score": "High",                                                                                              
     "score_explanation": "The CSR effectively resolved the customer's issue by processing full refunds for the    
 canceled trip in a timely manner."                                                                                
   },                                                                                                              
   "Product_Knowledge": {                                                                                          
     "score": "High",                                                                                              
     "score_explanation": "The CSR demonstrated a good understanding of the company's policies and procedures      
 regarding flight cancellations and refunds."                                                                      
   },                                                                                                              
   "Professionalism": {                                                                                            
     "score": "High",                                                                                              
     "score_explanation": "The CSR maintained a professional and courteous tone throughout the call, even when the 
 customer was upset."                                                                                              
   },                                                                                                              
   "Problem_Escalation": {                                                                                         
     "score": "High",                                                                                              
     "score_explanation": "The CSR recognized when the issue required escalation to a supervisor and transferred t 
 call smoothly."                                                                                                   
   },                                                                                                              
   "Resolution_Follow_Up": {                                                                                       
     "score": "Medium",                                                                                            
     "score_explanation": "The CSR did not explicitly mention any follow-up actions, but did inquire about the     
 customer's satisfaction with the resolution."                                                                     
   },                                                                                                              
   "Efficiency": {                                                                                                 
     "score": "High",                                                                                              
     "score_explanation": "The call was handled efficien

In [67]:
# Construct the call summary as JSON object with all relevant attributes and save it to S3
bedrock_response = json.loads(call_assessment.json())
bedrock_response ["call_ID"] = transcript_dict['call_ID']
bedrock_response ["CSR_ID"] = transcript_dict['CSR_ID']
bedrock_response ["call_date"] = call_date
bedrock_response ["call_time"] = call_time
bedrock_response ["llm_model"] = llm_modelId
bedrock_response = json.dumps (bedrock_response)
pretty_print_json(bedrock_response)

# Write Bedrock output text to S3 object  
if s3_bucket != "your-bucket-name":
    s3_key = s3_key_score + "Call score " + call_date + " " + call_time + ".json"
    s3.put_object(Body=bedrock_response, Bucket=s3_bucket, Key=s3_key )
    print("Transcript score assessment written to S3:" + s3_key)

else:
    rprint("[magenta bold]WARNING: No S3 bucket defined. Transcript score assessment not written to S3.[/magenta bold]")

                                                                                                                   
 {                                                                                                                 
   "Communication_Skills": {                                                                                       
     "score": "High",                                                                                              
     "score_explanation": "The CSR communicated clearly, listened actively to the customer's concerns, and         
 demonstrated empathy throughout the call."                                                                        
   },                                                                                                              
   "Problem_Resolution": {                                                                                         
     "score": "High",                                                                                              
     "score_explanation": "The CSR effectively resolved the customer's issue by processing full refunds for the    
 canceled trip in a timely manner."                                                                                
   },                                                                                                              
   "Product_Knowledge": {                                                                                          
     "score": "High",                                                                                              
     "score_explanation": "The CSR demonstrated a good understanding of the company's policies and procedures      
 regarding flight cancellations and refunds."                                                                      
   },                                                                                                              
   "Professionalism": {                                                                                            
     "score": "High",                                                                                              
     "score_explanation": "The CSR maintained a professional and courteous tone throughout the call, even when the 
 customer was upset."                                                                                              
   },                                                                                                              
   "Problem_Escalation": {                                                                                         
     "score": "High",                                                                                              
     "score_explanation": "The CSR recognized when the issue required escalation to a supervisor and transferred t 
 call smoothly."                                                                                                   
   },                                                                                                              
   "Resolution_Follow_Up": {                                                                                       
     "score": "Medium",                                                                                            
     "score_explanation": "The CSR did not explicitly mention any follow-up actions, but did inquire about the     
 customer's satisfaction with the resolution."                                                                     
   },                                                                                                              
   "Efficiency": {                                                                                                 
     "score": "High",                                                                                              
     "score_explanation": "The call was handled efficien

WARNING: No S3 bucket defined. Transcript score assessment not written to S3.

## The End of the Notebook